In [1]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-

from pymongo import MongoClient
from nltk.tokenize import TweetTokenizer,RegexpTokenizer

from stop_words import get_stop_words
import snowballstemmer
from gensim import corpora, models
import gensim
import pandas as pd

import snowballstemmer
stemmer = snowballstemmer.stemmer('Spanish')
stemmer.stemWord("llorar")
'llor'

In [2]:
try:
    client = MongoClient()
    print("Connected to MongoDB\n")
except pymongo.errors.ConnectionFailure as e:
    print("Could not connect to MongoDB",e)

Connected to MongoDB



In [3]:
db = client.sept19_26_keywords_db
tweets = db.sept19_26_keywords_collection

In [4]:
tweets.find().count()

40665

In [5]:
tknzr = TweetTokenizer(preserve_case=False,       # Convertir a minúsculas
                       reduce_len=True,           # Reducir caracteres repetidos
                       strip_handles=False)       # Mostrar @usuarios

stmmr = snowballstemmer.stemmer('Spanish')

In [6]:
s0 = "llueve otra vez silvio rodríguez vía @usuario mi canción favorita del gran silviorodríguez"
f=tknzr.tokenize(s0)

In [7]:
s0 = [stmmr.stemWord(i) for i in f]

In [8]:
s0

['lluev',
 'otra',
 'vez',
 'silvi',
 'rodriguez',
 'via',
 '@usuari',
 'mi',
 'cancion',
 'favorit',
 'del',
 'gran',
 'silviorodriguez']

In [9]:
spec_chars = ["…",'"',"“","/","(",")","[","]","?","¿","!","¡",
                 "rt",":","…",",","\n","#","\t","",".","$",
                 "...","-","🤢"]

In [13]:
def clean_tweet(tmp_tweet):
    """
    Eliminar tokens que:
    - Estén dentro de lista_de_paro.
    - Sean ligas.
    - Si es una mención i.e @potus, se cambia por token genérico @usuario.
    """
    return [i if i.startswith(('@')) else stmmr.stemWord(i) for i in tmp_tweet 
            if i not in spec_chars 
            and not i.startswith(('http', 'htt'))]

     

In [14]:
nuevos_tweets = set()
for i in tweets.find():
    if "retweeted_status" in i:           # Si es retweet...
        tmp = tknzr.tokenize(i["retweeted_status"]['text'])
        tmp1 = clean_tweet(tmp)
        nuevos_tweets,add(' '.join(tmp1))

    else:                                # Si no es retweet...
        tmp = tknzr.tokenize(i['text'])
        tmp = clean_tweet(tmp)
        nuevos_tweets.add(' '.join(tmp))


In [15]:
df = pd.DataFrame()

In [16]:
df["tweet"] = list(nuevos_tweets)

In [17]:
df.head(100)

,tweet
0,@chavarriaobras que tal buen dias podr ven a r...
1,conclu la jorn de acopi del dia de hoy lastim ...
2,centr de acopi en 20 de agost #50 coyoacan a u...
3,en el centr de acopi #cinemacoyoac necesit lec...
4,⒈ #sism ⒉ #prayformex ⒊ xochimilc ⒋ condes ⒌ #...
5,#sntesolidari el president nacional del #snte ...
6,ese horribl sentimient de vaci impotent y carg...
7,@chuckpee exact ahorit es moment de dej de per...
8,@edelamadrid ayud amig con vid atrap coquimb 9...
9,@ayudasismo2017 sab dond necesit un doctor fav...


In [ ]:
lda = LdaModel(list(nuevos_tweets),num_topics=50)